

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/NER_HEALTHCARE_DE.ipynb)




# **Detect symptoms, treatments and other NERs in German**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.



## 1. Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


## 2. Construct the pipeline

For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [10]:
document_assembler = DocumentAssembler() \
              .setInputCol("text")\
              .setOutputCol("document")

sentence_detector = SentenceDetector() \
              .setInputCols(["document"])\
              .setOutputCol("sentence")

tokenizer = Tokenizer()\
              .setInputCols(["sentence"]) \
              .setOutputCol("token")

# German word embeddings
word_embeddings = WordEmbeddingsModel.pretrained('w2v_cc_300d','de', 'clinical/models') \
              .setInputCols(["sentence", 'token'])\
              .setOutputCol("embeddings")

# German NER model
# Use only if you have w2v_cc_300d Word embeddings
clinical_ner = MedicalNerModel.pretrained("ner_healthcare_slim",'de','clinical/models') \
              .setInputCols(["sentence", "token", "embeddings"]) \
              .setOutputCol("ner")

# Use only if you have embeddings_healthcare Word embeddings
# clinical_ner = MedicalNerModel.pretrained('ner_healthcare', "de", "clinical/models") \
#     .setInputCols(["sentence", "token", "embeddings"])\
#     .setOutputCol("ner")


ner_converter = NerConverterInternal()\
              .setInputCols(['sentence', 'token', 'ner']) \
              .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[document_assembler, 
                                sentence_detector,
                                tokenizer,
                                word_embeddings,
                                clinical_ner,
                                ner_converter])

w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_healthcare_slim download started this may take some time.
[OK!]


----

Make sure the **Annotator** storage ref should match the storage ref for the **Embeddings**

In [5]:
word_embeddings.getStorageRef()

'w2v_cc_300d'

In [6]:
clinical_ner.getStorageRef()

'w2v_cc_300d'

----

## 3. Create example inputs

In [7]:
# Enter examples as strings in this array
input_list = [
    """Ja das ist bekannt gewesen. Und die haben sich danach gerichtet. Hatte das Gefühl das er sich schon sehr wohl da gefühlt. Haben das auch von den Betreuern als Rückmeldung erhalten. Er ist zwar immer furchtbar aufgeregt wenn es dann los geht, aber wenn er dann da ist, freut er sich doch. Müssen zwar dann auch meist er sehr lange Autofahrt in Kauf nehmen da es das nicht in unserer Gegend gibt. Aber wenn er wenigstens daran eine Freude, warum nicht.
Leute, lest euch diesen Beitrag vollständig durch! Cosma und ich, wir haben unsere Erlebnisse gut dokumentiert. Wir waren am Anfang genauso ratlos, wie du (und noch andere, die hier dieselben Fragen stellem), die Diagnose ist für solche Sachen entweder Depressionen oder Burnout bzw. beides (wenn nichts Körperliches vorliegt). Manche haben noch zusätzlich Angstzustände oder Panikattacken. Die Diagnose sollte allerdings ein Neurologe/Psychiater stellen.""",
    """Das ist doch immer eine Überlegung wert. Bis vor kurzem war das sogar noch die Strategie ganzer Länder, u. a. GB. Also ich bin zum Besispiel einer von den süßen Opis, die derzeit bloß nicht an den Eiern gedrückt werden sollen. Ich würde mir tatsächlich eine Ansteckung mit einem einigermaßen moderaten Verlauf wünschen. Dann hätte ich, nach allem was man weiß, u. a. von Rhesusaffen, eine Grundimmunität und könnte rausgehen und meinen Mitmenschen unter die Arme greifen. Ich bin halt alt, aber net unbedingt schwach. Rüstiger Greis, sozusagen"""
]

## 4. Use the pipeline to create outputs

In [8]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(input_list, StringType()).toDF('text')
result = nlp_pipeline.fit(df).transform(df)

## 5. Visualize results

In [9]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)